In [ ]:
%matplotlib inline

# Load the machine
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
from util.simulation import *
from util.failures import *
from IPython.display import display, HTML

In [ ]:
# Chargement des machines
machine_ref = load(join(DATA_DIR, "Machine", "TESLA_S.json"))
machine_defect1 = load("Tesla_Model3_Variantes/Variantes_Defauts/Tesla_Model3_Defaut_Court_Circuit_Moins1_Tour.json")

In [ ]:
def check_machine_materials(machine):
    """Vérifie que tous les matériaux sont définis"""
    print("=== VÉRIFICATION DES MATÉRIAUX ===")
    print(f"Type de machine: {type(machine).__name__}")
    print(f"Type de rotor: {type(machine.rotor).__name__}")
    
    # Vérification  de stator
    if hasattr(machine.stator, 'mat_type'):
        print(f" Stator: {machine.stator.mat_type.name}")
    else:
        print(" Stator: Matériau manquant")
    
    # Vérification de rotor
    if hasattr(machine.rotor, 'mat_type'):
        print(f" Rotor: {machine.rotor.mat_type.name}")
    else:
        print(" Rotor: Matériau manquant")
    
    # Vérification des aimants 
    if hasattr(machine.rotor, 'hole'):
        print(" Rotor à aimants détecté")
        for i, hole in enumerate(machine.rotor.hole):
            for attr_name in dir(hole):
                if attr_name.startswith('magnet_'):
                    magnet = getattr(hole, attr_name)
                    if magnet is not None:
                        if hasattr(magnet, 'mat_type'):
                            print(f" Aimant {i}: {magnet.mat_type.name}")
                        else:
                            print(f" Aimant {i}: Matériau manquant")
    elif hasattr(machine.rotor, 'slot'):
        print(" Rotor à cage d'écureuil détecté")
        if hasattr(machine.rotor.slot, 'conductor'):
            if hasattr(machine.rotor.slot.conductor, 'cond_mat'):
                print(f" Conducteur rotor: {machine.rotor.slot.conductor.cond_mat.name}")
            else:
                print(" Conducteur rotor: Matériau manquant")
        else:
            print("  Pas de conducteur défini pour le rotor")
    
    # Vérification de bobinage stator
    if hasattr(machine.stator.winding, 'conductor'):
        if hasattr(machine.stator.winding.conductor, 'cond_mat'):
            print(f" Conducteur stator: {machine.stator.winding.conductor.cond_mat.name}")
        else:
            print(" Conducteur stator: Matériau manquant")
    else:
        print("  Pas de conducteur défini pour le stator")


check_machine_materials(machine_ref)

In [ ]:
def fix_squirrel_cage_materials(machine):
    """Corrige les matériaux pour machine à cage d'écureuil"""
    
    # Chargement des matériaux standards
    from pyleecan.Functions.load import load
    from pyleecan.definitions import DATA_DIR
    
    M400_50A = load(join(DATA_DIR, "Material", "M400-50A.json"))
    Copper1 = load(join(DATA_DIR, "Material", "Copper1.json"))
    
    print("=== CORRECTION DES MATÉRIAUX CAGE D'ÉCUREUIL ===")
    
    # Stator
    if not hasattr(machine.stator, 'mat_type') or machine.stator.mat_type is None:
        machine.stator.mat_type = M400_50A
        print(" Matériau stator assigné")
    
    # Rotor
    if not hasattr(machine.rotor, 'mat_type') or machine.rotor.mat_type is None:
        machine.rotor.mat_type = M400_50A
        print(" Matériau rotor assigné")
    
    # Conducteur rotor (cage d'écureuil)
    if hasattr(machine.rotor, 'slot'):
        print(" Configuration du conducteur rotor...")
        
        # Vérification de l'existance de onducteur
        if not hasattr(machine.rotor.slot, 'conductor'):
            # Création de conducteur
            from pyleecan.Classes.CondType11 import CondType11
            machine.rotor.slot.conductor = CondType11()
            print(" Conducteur rotor créé")
        
        # Assignement de matériau
        if not hasattr(machine.rotor.slot.conductor, 'cond_mat') or machine.rotor.slot.conductor.cond_mat is None:
            machine.rotor.slot.conductor.cond_mat = Copper1
            print(" Matériau conducteur rotor assigné")
        
        # Configuration des paramètres du conducteur
        if hasattr(machine.rotor.slot.conductor, 'Nwppc_tan'):
            machine.rotor.slot.conductor.Nwppc_tan = 1
        if hasattr(machine.rotor.slot.conductor, 'Nwppc_rad'):
            machine.rotor.slot.conductor.Nwppc_rad = 1
        if hasattr(machine.rotor.slot.conductor, 'Wwire'):
            machine.rotor.slot.conductor.Wwire = 0.001  # 1mm
        if hasattr(machine.rotor.slot.conductor, 'Hwire'):
            machine.rotor.slot.conductor.Hwire = 0.001  # 1mm
        
        print(" Paramètres conducteur rotor configurés")
    
    # Conducteur stator
    if hasattr(machine.stator.winding, 'conductor'):
        if not hasattr(machine.stator.winding.conductor, 'cond_mat') or machine.stator.winding.conductor.cond_mat is None:
            machine.stator.winding.conductor.cond_mat = Copper1
            print(" Matériau conducteur stator assigné")

# Corriger la machine
fix_squirrel_cage_materials(machine_ref)

In [ ]:
def verify_machine_completeness(machine):
    """Vérifie que la machine est complète pour la simulation"""
    
    print("=== VÉRIFICATION COMPLÈTUDE MACHINE ===")
    
    # Vérifier la géométrie
    try:
        fig, ax = machine.plot(is_show_fig=False)
        print("✅ Géométrie OK")
    except Exception as e:
        print(f"❌ Problème géométrie: {e}")
        return False
    
    # Vérifier les matériaux
    check_machine_materials(machine)
    
    # Vérifier la cohérence
    airgap = machine.stator.Rint - machine.rotor.Rext
    if airgap <= 0:
        print(f"❌ Entrefer négatif: {airgap*1000:.2f} mm")
        return False
    else:
        print(f"✅ Entrefer OK: {airgap*1000:.2f} mm")
    
    # Vérifier le bobinage
    if machine.stator.winding.Ntcoil <= 0:
        print(f"❌ Nombre de tours invalide: {machine.stator.winding.Ntcoil}")
        return False
    else:
        print(f"✅ Bobinage OK: {machine.stator.winding.Ntcoil} tours")
    
    print("✅ Machine prête pour simulation")
    return True

# Vérifier
verify_machine_completeness(machine_ref)

In [ ]:
# Charger les matériaux de référence
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR

# Charger les matériaux standards
M400_50A = load(join(DATA_DIR, "Material", "M400-50A.json"))
Copper1 = load(join(DATA_DIR, "Material", "Copper1.json"))

# Créer un matériau d'aimant si nécessaire
from pyleecan.Classes.Material import Material
from pyleecan.Classes.MatMagnetics import MatMagnetics

# Matériau d'aimant Tesla
Magnet_Tesla = Material(name="Magnet_Tesla")
Magnet_Tesla.mag = MatMagnetics(
    mur_lin=1.05,
    alpha_Br=-0.001,
    Brm20=1.24,
    Wlam=0
)
Magnet_Tesla.elec.rho = 1.6e-06
Magnet_Tesla.struct.rho = 7500.0

# Assigner les matériaux à la machine
def fix_machine_materials(machine):
    """Corrige les matériaux manquants"""
    
    # Stator
    if not hasattr(machine.stator, 'mat_type') or machine.stator.mat_type is None:
        machine.stator.mat_type = M400_50A
        print(" Matériau stator assigné")
    
    # Rotor
    if not hasattr(machine.rotor, 'mat_type') or machine.rotor.mat_type is None:
        machine.rotor.mat_type = M400_50A
        print(" Matériau rotor assigné")
    
    # Aimants
    for hole in machine.rotor.hole:
        for attr_name in dir(hole):
            if attr_name.startswith('magnet_'):
                magnet = getattr(hole, attr_name)
                if magnet is not None:
                    if not hasattr(magnet, 'mat_type') or magnet.mat_type is None:
                        magnet.mat_type = Magnet_Tesla
                        print(f" Matériau aimant {attr_name} assigné")
    
    # Conducteur
    if hasattr(machine.stator.winding, 'conductor'):
        if not hasattr(machine.stator.winding.conductor, 'cond_mat') or machine.stator.winding.conductor.cond_mat is None:
            machine.stator.winding.conductor.cond_mat = Copper1
            print(" Matériau conducteur assigné")

# Corriger la machine
fix_machine_materials(machine_ref)

In [ ]:
simu_ref = load_simulation(name="ref", machine=machine_ref, rotor_speed=3000, stop=1, num_steps=4000)
simu_defect1 = load_simulation(name="defect1", machine=machine_defect1, rotor_speed=3000, stop=1, num_steps=4000)
simu_defect2 = load_simulation(name="defect2", machine=machine_defect2, rotor_speed=3000, stop=1, num_steps=4000)

In [ ]:
out_ref = simu_ref.run()
out_defect1 = simu_defect1.run()
out_defect2 = simu_defect2.run()

compare_simulation_results(
    out1=out_ref,
    out2=out_defect1,
    legend_list=["Toyota_Prius_Ref", "Toyota_Prius_defect_1_Ntcoil_7"]
)
compare_simulation_results(
    out1=out_ref,
    out2=out_defect2,
    legend_list=["Toyota_Prius_Ref", "Toyota_Prius_defect_2_Ntcoil_7"]
)
